# See Spot. See Spot Run.

What follows is an introduction to the Spot issue spotter API avilable at [https://spot.suffolklitlab.org/](https://spot.suffolklitlab.org/). Give Spot a non-lawyer's description of a situation, and it returns a list of likely issues from the National Subject Matter Index (NSMI), [Version 2](https://spot.suffolklitlab.org/performance/#nsmi). The NSMI provides the legal aid community with a standard nomenclature for talking about client needs. It includes issues like eviction, foreclosure, bankruptcy, and child support. Spot is provided as a service over an API. Mostly, this means it's built for use by computer programs, not people. Coders can build things (like websites) on top of the API. The hope is that by automating part of issue identification, developers will use Spot to help people in need of legal assistance better access available resources. See [Pew Grant Will Take ‘Learned Hands’ Project from Prototype to Production, to Help ID Consumers’ Legal Issues](https://www.lawsitesblog.com/2019/07/pew-grant-will-take-learned-hands-project-from-prototype-to-production-to-help-id-consumers-legal-issues.html).

You'll want to first review the API's documenation here: https://app.swaggerhub.com/apis-docs/suffolklitlab/spot/

Then you'll want to create an account here and get your user token. https://spot.suffolklitlab.org/user/new/

Once, you have a token, you can use it below to explore the API.

## Load libraries and set your user token

Note: If you every just want to play around with a RESTful API in your browser, the RESTED plugin is a useful tool. See [Chrome](https://chrome.google.com/webstore/detail/rested/eelcnbccaccipfolokglfhhmapdchbfg?hl=en-US) and [Firefox](https://addons.mozilla.org/en-US/firefox/addon/rested/) implemnations. 

In [ ]:
import requests
import json

# replace with your own token
token = ""

headers = { "Authorization": "Bearer " + token, 
            "Content-Type":"application/json" }

## Prepare and run your query

The API returns a JSON object. If you need help viewing it, consider cutting and pasting the content into this tool. http://jsonviewer.stack.hu/

In [ ]:
body = {
  "text": "I was kicked out of my apartment.",
  "save-text": 1,
  "cutoff-lower": 0.25,
  "cutoff-pred": 0.5,
  "cutoff-upper": 0.5
}

r = requests.post('https://spot.suffolklitlab.org/v0/entities-nested/', headers=headers, data=json.dumps(body))
output_ = r.json()

output_

## Clean up the reply

Let's use a for loop to run through the output. 

In [ ]:
print("Your text:")
print(output_["text"])
print("\nPossible issues:")
for item in output_["labels"]:
    print("-",item["name"])
    try:
        for child in item["children"]:
            print("--",child["name"])
    except:
        print("-- NO CHILD ISSUES")

Now let's add a bit to show the lower and upper bounds of the predictions. 

In [ ]:
print("Your text:")
print(output_["text"])
print("\nPossible issues:")
for item in output_["labels"]:
    print("-",item["name"],"(%s%%-%s%%)"%(round(item["lower"]*100),round(item["upper"]*100)))
    try:
        for child in item["children"]:
            print("--",child["name"],"(%s%%-%s%%)"%(round(child["lower"]*100),round(child["upper"]*100)))
    except:
        print("-- NO CHILD ISSUES")

Now let's make this all into a single function

In [ ]:
def getNSMI(token,text,save=1,lower=0.25,pred=0.5,upper=0.5):
    
    headers = { "Authorization": "Bearer " + token, 
                "Content-Type":"application/json" }

    body = {
      "text": text,
      "save-text": save,
      "cutoff-lower": lower,
      "cutoff-pred": pred,
      "cutoff-upper": upper
    }

    r = requests.post('https://spot.suffolklitlab.org/v0/entities-nested/', headers=headers, data=json.dumps(body))
    output_ = r.json()

    print("Your text:")
    print(output_["text"])
    print("\nPossible issues:")
    for item in output_["labels"]:
        print("-",item["name"],"(%s%%-%s%%)"%(round(item["lower"]*100),round(item["upper"]*100)))
        try:
            for child in item["children"]:
                print("--",child["name"],"(%s%%-%s%%)"%(round(child["lower"]*100),round(child["upper"]*100)))
        except:
            print("-- NO CHILD ISSUES")
    
    return output_

In [ ]:
text = input("What seems to be the issue? ")
output_ = getNSMI(token,text,lower=0.25)